In [1]:
import pandas as pd
import numpy as np

In [2]:
import dask
from dask.distributed import Client
from dask import dataframe as dd
import sys

print("Python Version : ", sys.version)
print("Dask Version : ", dask.__version__)

client = Client(n_workers=2, threads_per_worker=4)
client

Python Version :  3.9.7 (default, Sep 16 2021, 13:09:58) 
[GCC 7.5.0]
Dask Version :  2021.12.0


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 2
Total threads: 8,Total memory: 35.19 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:44739,Workers: 2
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 35.19 GiB
Comm: tcp://127.0.0.1:40533,Total threads: 4
Dashboard: http://127.0.0.1:39319/status,Memory: 17.59 GiB
Nanny: tcp://127.0.0.1:43751,


### Data

In [3]:
data_len = 10000000
id_list1 = [i for i in range(data_len)]
id_list2 = [i for i in range(data_len,2*data_len)]
df1 = pd.DataFrame({'id': id_list1,
                    'value': np.random.randint(0,100,data_len)})
df2 = pd.DataFrame({'id': id_list2,
                    'value': np.random.randint(0,100,data_len)})

### Pandas

In [4]:
%time df1.merge(df2, on ='id', how='left')

CPU times: user 2.79 s, sys: 892 ms, total: 3.69 s
Wall time: 3.57 s


,id,value_x,value_y
0,0,1,NaN
1,1,36,NaN
2,2,77,NaN
3,3,25,NaN
4,4,46,NaN
...,...,...,...
9999995,9999995,80,NaN
9999996,9999996,69,NaN
9999997,9999997,97,NaN
9999998,9999998,82,NaN


### Dask

In [5]:
dd1 = dask.dataframe.from_pandas(df1, npartitions=2)
dd2 = dask.dataframe.from_pandas(df2, npartitions=2)

In [6]:
%time dd1.merge(dd2, on ='id', how='left').compute()

CPU times: user 979 ms, sys: 920 ms, total: 1.9 s
Wall time: 6.79 s


,id,value_x,value_y
0,1,36,NaN
1,9,38,NaN
2,10,41,NaN
3,11,31,NaN
4,13,78,NaN
...,...,...,...
5001474,9999984,20,NaN
5001475,9999988,65,NaN
5001476,9999991,19,NaN
5001477,9999992,76,NaN
